In [22]:
#Importing the required libraries

import pandas as pd

In [23]:
#Loading the CSV files containing the 10-k data of the 3 companies from the last 3 fiscal years

df = pd.read_csv("/content/Financial_Data_Microsoft_Tesla_Apple.csv")
df

,Company,Fiscal Year,Total Revenue (in millions USD),Net Income (in millions USD),Total Assets (in millions USD),Total Liabilities (in millions USD),Cash Flow from Operating Activities (in millions USD)
0,Microsoft,2023,232319,72361,411979,198298,87596
1,Microsoft,2022,198270,72738,364840,198298,89036
2,Microsoft,2021,168088,61271,333779,191791,76737
3,Tesla,2023,96773,15015,102513,52733,13879
4,Tesla,2022,81462,12556,82338,39485,14724
5,Tesla,2021,53823,5519,62131,30548,11497
6,Apple,2023,383285,97000,352583,290437,110543
7,Apple,2022,394328,99803,351002,302083,122151
8,Apple,2021,365817,94680,351000,287912,104038


In [24]:
df.rename(columns={
    'Fiscal Year': 'Year',
    'Total Revenue (in millions USD)': 'Total Revenue',
    'Net Income (in millions USD)': 'Net Income',
    'Total Assets (in millions USD)': 'Total Assets',
    'Total Liabilities (in millions USD)': 'Total Liabilities',
    'Cash Flow from Operating Activities (in millions USD)': 'Cash Flow from Operating Activities'
}, inplace=True)

print(df.columns)

Index(['Company', 'Year', 'Total Revenue', 'Net Income', 'Total Assets',
       'Total Liabilities', 'Cash Flow from Operating Activities'],
      dtype='object')


In [25]:
# Calculate year-over-year growth rates for Total Revenue and Net Income
df['Revenue Growth (%)'] = df.groupby('Company')['Total Revenue'].pct_change() * 100
df['Net Income Growth (%)'] = df.groupby('Company')['Net Income'].pct_change() * 100

# Fill NA values that result from pct_change calculations with 0 or an appropriate value
df.fillna(0, inplace=True)

# Display the dataframe to verify the calculations
print(df)

# Optionally, you could summarize these findings for each company
summary = df.groupby('Company').agg({
    'Revenue Growth (%)': 'mean',
    'Net Income Growth (%)': 'mean'
}).reset_index()

print("\nYear-over-Year Average Growth Rates (%):")
print(summary)

     Company  Year  Total Revenue  Net Income  Total Assets  \
0  Microsoft  2023         232319       72361        411979   
1  Microsoft  2022         198270       72738        364840   
2  Microsoft  2021         168088       61271        333779   
3      Tesla  2023          96773       15015        102513   
4      Tesla  2022          81462       12556         82338   
5      Tesla  2021          53823        5519         62131   
6      Apple  2023         383285       97000        352583   
7      Apple  2022         394328       99803        351002   
8      Apple  2021         365817       94680        351000   

   Total Liabilities  Cash Flow from Operating Activities  Revenue Growth (%)  \
0             198298                                87596            0.000000   
1             198298                                89036          -14.656141   
2             191791                                76737          -15.222676   
3              52733                         

In [26]:
def simple_chatbot(user_query, company, year):

    # Filtering the dataframe
    data = df[(df['Company'].str.lower() == company.lower()) & (df['Year'] == year)]

    if data.empty:
        return "Sorry, no data available for that company and year."

    row = data.iloc[0]

    if user_query == "What is the total revenue?":
        return f"The total revenue for {company} in {year} was ${row['Total Revenue']} million."
    elif user_query == "How has net income changed over the last year?":
        prev_data = df[(df['Company'].str.lower() == company.lower()) & (df['Year'] == year - 1)]
        if prev_data.empty:
            return "No previous year data available to compare net income."
        diff = row['Net Income'] - prev_data.iloc[0]['Net Income']
        trend = "increased" if diff > 0 else "decreased"
        return f"The net income has {trend} by ${abs(diff)} million from {year - 1} to {year}."
    elif user_query == "What is the total assets?":
        return f"The total assets for {company} in {year} were ${row['Total Assets']} million."
    elif user_query == "What is the total liabilities?":
        return f"The total liabilities for {company} in {year} were ${row['Total Liabilities']} million."
    elif user_query == "How has cash flow from operating activities changed over the last year?":
        prev_data = df[(df['Company'].str.lower() == company.lower()) & (df['Year'] == year - 1)]
        if prev_data.empty:
            return "No previous year data available to compare cash flow."
        diff = row['Cash Flow from Operating Activities'] - prev_data.iloc[0]['Cash Flow from Operating Activities']
        trend = "increased" if diff > 0 else "decreased"
        return f"The cash flow from operating activities has {trend} by ${abs(diff)} million from {year - 1} to {year}."
    else:
        return "Sorry, I can only provide information on predefined queries."


In [27]:
company = input("Enter company name: ")
year = int(input("Enter fiscal year (e.g., 2023): "))
query = input("Enter your financial question: ")

response = simple_chatbot(query, company, year)
print("Chatbot:", response)


Enter company name: Apple
Enter fiscal year (e.g., 2023): 2023
Enter your financial question: How has net income changed over the last year?
Chatbot: The net income has decreased by $2803 million from 2022 to 2023.
